In [1]:
import datetime as dt
import random
import tensorflow as tf

# Get timestamp
timestamp = dt.datetime.now().strftime('%Y%m%d%H%M%S')

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.1
training_epochs = 8000
batch_size = 100
display_step = 10

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# tf graph input
X = tf.placeholder(tf.float32, [None, 784], name = 'X-input')
Y = tf.placeholder(tf.float32, [None, 10], name = 'Y-input')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name = 'Weight')
B = tf.Variable(tf.zeros([10]), name = 'Bias')

tf.histogram_summary('Weight', W)
tf.histogram_summary('Bias', B)

<tf.Tensor 'HistogramSummary_1:0' shape=() dtype=string>

In [3]:
# Construct model
with tf.name_scope('Layer'):
    hypothesis = tf.nn.softmax(tf.matmul(X, W) + B)

# Minimize error using cross entropy
with tf.name_scope('Cost'):
    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices = 1))
    tf.scalar_summary('Cost', cost)

# Gradient Descent
with tf.name_scope('Train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Accuracy
with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1)), tf.float32))
    tf.scalar_summary('Accuracy', accuracy)

In [4]:
# Launch the graph
with tf.Session() as sess:
    writer = tf.train.SummaryWriter('/logs/mnist/softmax_' + timestamp, sess.graph)
    merged = tf.merge_all_summaries()
    
    tf.initialize_all_variables().run()

    # Training cycle
    for epoch in range(training_epochs):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # Fit training using batch data
        _, summary = sess.run([optimizer, merged], feed_dict = {X: batch_xs, Y: batch_ys})

        # Summarize
        writer.add_summary(summary, epoch)

    writer.close()
    print('Accuracy', accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

Accuracy 0.9216
